# AQM vs CCA Experiments on FABRIC
Create a two-site topology (h1–R1–R2–h2), configure hosts and routers, run iPerf3 experiments, and plot results.


## Overview

This notebook is a reproducible runbook for evaluating Active Queue Management (AQM) and congestion control behavior on a controlled FABRIC topology.

**You will:**
* Provision a two-site slice with two routers and two host VMs.
* Optionally install experimental kernels (BBRv3 or Prague).
* Launch Mininet topologies on the host VMs.
* Run parallel iPerf3 transfers and collect JSON logs.
* Plot throughput over time.

**Important notes:**
* Some sections reboot nodes. Run those sections only when needed.
* Long-running Mininet and iPerf3 processes are best started from SSH sessions, not from the notebook kernel.


## Topology


<img src="../AQM_CCA/files/topo.png" width="850px" alt="Topology diagram showing hosts h1 and h2 connected through routers R1 and R2, with Mininet host groups on each side."><br>

**High-level flow:** Mininet senders on **h1** → **R1** (bottleneck and AQM/buffer settings) → **R2** → Mininet receivers on **h2**.


## 1. Prerequisites and environment setup

Before running this notebook, configure your FABRIC environment using the **Configure Environment** notebook:
`../../../configure_and_validate.ipynb`.

* If you are using **FABRIC JupyterHub**, most environment variables are already set. You still need your bastion username and a bastion private key file.
* If you are using the **FABRIC API outside JupyterHub**, confirm all environment variables in your shell or notebook kernel.

If you run into bastion or SSH issues, the FABRIC knowledge base and the FABRIC user forum are the fastest place to troubleshoot.


## 2. Import libraries and initialize FABlib

This section imports the FABlib client used to create, inspect, and manage FABRIC slices.


In [1]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

## 3. Create the FABRIC slice

This section defines the experiment resources (nodes and L2 networks), then submits a slice request for instantiation.


### 3.1 Configure slice parameters

Edit the parameters in the next cell to match your experiment needs (sites, image, cores, RAM, addressing plan, and network names).


In [2]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'AQM_CCA'

#[site1,site2] = fablib.get_random_sites(count=2)
site1="WASH"
site2="LOSA"
print(f"Sites: {site1},{site2}")

# Routers
R1_name = "R1"
R2_name = "R2"

router_cores = 8
router_ram = 16
router_disk = 20

# Hosts
h1_name = "h1"
h2_name = "h2"

h1_subnet=IPv4Network('172.16.0.0/16')
h1_addr=IPv4Address('172.16.255.10')
R1_addr1=IPv4Address('172.16.255.254')

h2_subnet=IPv4Network('172.17.0.0/16')
h2_addr=IPv4Address('172.17.255.10')
R2_addr1=IPv4Address('172.17.255.254')

R1_subnet=IPv4Network('192.168.12.0/30')
R1_addr2=IPv4Address('192.168.12.1')

R2_subnet=IPv4Network('192.168.12.0/30')
R2_addr2=IPv4Address('192.168.12.2')

host_cores = 16
host_ram = 32
host_disk = 20

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'

net_R1_R2_name = 'net_R1_R2'

# All node properties
image = 'default_ubuntu_24'


Sites: WASH,LOSA


### 3.2 Build and submit the slice

This step builds the slice object (nodes, NICs, and L2 networks) and submits it for provisioning.
Provisioning can take several minutes depending on site availability.


In [3]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add router node R1
    R1 = slice.add_node(name=R1_name, site=site1,  image=image, 
                        cores=router_cores, ram=router_ram, disk=router_disk)
    R1.set_capacities(cores=router_cores, ram=router_ram, disk=router_disk)
    R1_iface_1 = R1.add_component(model='NIC_Basic', name="R1_router_nic1").get_interfaces()[0]
    R1_iface_2 = R1.add_component(model='NIC_Basic', name="R1_router_nic2").get_interfaces()[0]

     # Add router node R1
    R2 = slice.add_node(name=R2_name, site=site2,  image=image, 
                        cores=router_cores, ram=router_ram, disk=router_disk)
    R2.set_capacities(cores=router_cores, ram=router_ram, disk=router_disk)
    R2_iface_1 = R2.add_component(model='NIC_Basic', name="R2_router_nic1").get_interfaces()[0]
    R2_iface_2 = R2.add_component(model='NIC_Basic', name="R2_router_nic2").get_interfaces()[0]
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    
    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[h1_iface, R1_iface_1])
    router_net1 = slice.add_l2network(name=net_R1_R2_name, interfaces=[R1_iface_2, R2_iface_2])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[h2_iface, R2_iface_1])
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()


Retry: 11, Time: 265 sec


ID,59986642-3002-4ddc-9e4c-f0e1edab1ddd
Name,AQM_CCA
Lease Expiration (UTC),2026-02-15 16:17:13 +0000
Lease Start (UTC),2026-02-14 16:17:13 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK
Email,jagomez@fortlewis.edu
UserId,b884fd22-4166-4c28-bb4d-ea7b81fb5f3c


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
af608029-c99b-4627-b927-caab36d8f910,R1,8,16,100,default_ubuntu_24,qcow2,wash-w3.fabric-testbed.net,WASH,ubuntu,2001:400:a100:3020:f816:3eff:fee6:96b5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fee6:96b5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f41b8a42-dbf5-4e48-827d-776d8be6c3b1,R2,8,16,100,default_ubuntu_24,qcow2,losa-w1.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe0f:fa79,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe0f:fa79,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5725ab6a-a783-45a9-b933-5bf07826291e,h1,16,32,100,default_ubuntu_24,qcow2,wash-w3.fabric-testbed.net,WASH,ubuntu,2001:400:a100:3020:f816:3eff:fe25:e031,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe25:e031,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c6a61214-9133-4862-8bb0-2cbb9e20266d,h2,16,32,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe31:fe2d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe31:fe2d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
959d8ed0-d223-445d-9620-33045339d24b,net_R1_R2,L2,L2STS,None,None,None,Active,
bf74105f-1cb2-4a6a-9001-abdcd5f2b384,net_h1,L2,L2Bridge,WASH,None,None,Active,
239318f6-a9d1-48ef-af20-8437ce2a473a,net_h2,L2,L2Bridge,LOSA,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
R1-R1_router_nic1-p1,p1,R1,net_h1,100,config,,3E:15:F2:7B:E0:82,enp7s0,enp7s0,fe80::3c15:f2ff:fe7b:e082,4,HundredGigE0/0/0/9
R1-R1_router_nic2-p1,p1,R1,net_R1_R2,100,config,,3E:24:05:44:96:74,enp8s0,enp8s0,fe80::3c24:5ff:fe44:9674,4,HundredGigE0/0/0/9
R2-R2_router_nic1-p1,p1,R2,net_h2,100,config,,02:ED:A5:B6:72:0D,enp7s0,enp7s0,fe80::ed:a5ff:feb6:720d,6,HundredGigE0/0/0/5
R2-R2_router_nic2-p1,p1,R2,net_R1_R2,100,config,,02:4C:5D:8A:AD:A9,enp6s0,enp6s0,fe80::4c:5dff:fe8a:ada9,6,HundredGigE0/0/0/5
h1-h1_nic-p1,p1,h1,net_h1,100,config,,42:24:95:57:DD:DF,enp7s0,enp7s0,fe80::4024:95ff:fe57:dddf,4,HundredGigE0/0/0/9
h2-h2_nic-p1,p1,h2,net_h2,100,config,,06:06:AA:6A:FB:71,enp6s0,enp6s0,fe80::406:aaff:fe6a:fb71,4,HundredGigE0/0/0/9



Time to print interfaces 272 seconds


### 3.3 Install a TCP BBRv3 kernel on h1

Use this section only if you need the custom BBRv3 kernel for experiments.
It installs kernel packages and reboots the node.
After the reboot, you may need to reconnect to the slice and rerun any configuration steps that depend on an active SSH session.


In [4]:
### 3.4 Install TCP Prague (L4S kernel build) on h2
h1 = slice.get_node(name="h1")     
h1.upload_file('kernel_module/linux-headers-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb', 'linux-headers-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb')
h1.upload_file('kernel_module/linux-image-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb', 'linux-image-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb')
h1.execute("sudo dpkg -i linux-headers-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb")
h1.execute("sudo dpkg -i linux-image-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb")
h1.execute("echo 'net.ipv4.tcp_congestion_control=bbr' | sudo tee -a /etc/sysctl.conf")
h1.execute("sudo rm *.deb")
h1.execute("sudo sysctl -p")
#h1.execute("sysctl net.ipv4.tcp_congestion_control")
h1.execute("sudo reboot")

Selecting previously unselected package linux-headers-6.4.0-bbrv3.
(Reading database ... 74797 files and directories currently installed.)
Preparing to unpack linux-headers-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb ...
Unpacking linux-headers-6.4.0-bbrv3 (6.4.0-g7542cc7c41c0-1) ...
Setting up linux-headers-6.4.0-bbrv3 (6.4.0-g7542cc7c41c0-1) ...
Selecting previously unselected package linux-image-6.4.0-bbrv3.
(Reading database ... 85045 files and directories currently installed.)
Preparing to unpack linux-image-6.4.0-bbrv3_6.4.0-g7542cc7c41c0-1_amd64.deb ...
Unpacking linux-image-6.4.0-bbrv3 (6.4.0-g7542cc7c41c0-1) ...
Setting up linux-image-6.4.0-bbrv3 (6.4.0-g7542cc7c41c0-1) ...
update-initramfs: Generating /boot/initrd.img-6.4.0-bbrv3
Sourcing file `/etc/default/grub'
Sourcing file `/etc/default/grub.d/50-cloudimg-settings.cfg'
Generating grub configuration file ...
Found linux image: /boot/vmlinuz-6.8.0-63-generic
Found initrd image: /boot/initrd.img-6.8.0-63-generic
Found linux 

('', '')

### 3.4 Install TCP Prague (L4S kernel build)

Use this section only if you need TCP Prague and DualPI2 support for L4S experiments.
It downloads the L4S team build, installs packages, updates GRUB, and reboots the node.


In [6]:
h1 = slice.get_node(name="h1")     
h1.execute("wget https://github.com/L4STeam/linux/releases/download/l4steam-6.12.y-build/l4s-l4steam-6.12.y.zip &> /dev/null")
h1.execute("sudo apt -y install unzip")
h1.execute("unzip l4s-l4steam-6.12.y.zip")
h1.execute("sudo dpkg --install debian_build/*")
h1.execute("sudo update-grub")
h1.execute("sudo reboot")
h1.execute("uname -r")
h1.execute("sudo modprobe sch_dualpi2")
h1.execute("sudo modprobe tcp_prague")
h1.execute("sudo sysctl net.ipv4.tcp_available_congestion_control")

h2 = slice.get_node(name="h2")   
h2.execute("wget https://github.com/L4STeam/linux/releases/download/l4steam-6.12.y-build/l4s-l4steam-6.12.y.zip &> /dev/null")
h2.execute("sudo apt -y install unzip")
h2.execute("unzip l4s-l4steam-6.12.y.zip")
h2.execute("sudo dpkg --install debian_build/*")
h2.execute("sudo update-grub")
h2.execute("sudo reboot")
h2.execute("uname -r")
h2.execute("sudo modprobe sch_dualpi2")
h2.execute("sudo modprobe tcp_prague")
h2.execute("sudo sysctl net.ipv4.tcp_available_congestion_control")




Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 174 kB of archives.
After this operation, 384 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 unzip amd64 6.0-28ubuntu4.1 [174 kB]
debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Fetched 174 kB in 1s (255 kB/s)
Selecting previously unselected package unzip.
(Reading database ... 74797 files and directories currently installed.)
Preparin

('net.ipv4.tcp_available_congestion_control = reno cubic prague\n', '')

### 3.5 Verify kernel and congestion control availability

Confirm kernel version and available congestion control modules before moving on.


In [7]:
h1.execute("uname -a")
h1.execute("sysctl net.ipv4.tcp_available_congestion_control")
h2.execute("uname -a")
h2.execute("sysctl net.ipv4.tcp_available_congestion_control")

Linux h1 6.12.54-0b264c55b-l4steam-117 #1 SMP PREEMPT_DYNAMIC Fri Nov  7 18:07:57 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
net.ipv4.tcp_available_congestion_control = reno cubic bbr prague
Linux h2 6.12.54-0b264c55b-l4steam-117 #1 SMP PREEMPT_DYNAMIC Fri Nov  7 18:07:57 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
net.ipv4.tcp_available_congestion_control = reno cubic prague


('net.ipv4.tcp_available_congestion_control = reno cubic prague\n', '')

## 4. Configure nodes (packages, IPs, and routing)

This section prepares the VMs so they can run Mininet topologies and iPerf3 experiments.


### 4.1 Install packages and assign addresses

Installs common tooling (iperf3, net-tools, Mininet dependencies, plotting libraries), sets hostnames, and assigns the experiment IP addresses to FABRIC interfaces.


In [8]:
config_threads = {}

In [19]:
host_config_script = "sudo apt-get update -qq && sudo apt-get -y install && sudo apt-get -y install net-tools -qq && sudo apt-get -y install iperf3 -qq  && sudo apt-get -y install mininet -qq && sudo apt-get -y install python3-matplotlib -qq" 
h1.execute("sudo apt -y install --fix-broken")
h2.execute("sudo apt install --fix-broken")
try: 
    h1 = slice.get_node(name=h1_name)
    h1.execute("sudo sed -i '1s/.*/127.0.0.1 localhost h1/' /etc/hosts")
    if type(ip_address(h1.get_management_ip())) is IPv6Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")   
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
    h1_os_iface.ip_addr_add(addr=h1_addr, subnet=h1_subnet)
    h1_config_thread = h1.execute_thread(host_config_script)
    config_threads[h1] = h1_config_thread
    
    h2 = slice.get_node(name=h2_name)
    h2.execute("sudo sed -i '1s/.*/127.0.0.1 localhost h2/' /etc/hosts")
    if type(ip_address(h2.get_management_ip())) is IPv6Address:
        h2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.ip_addr_add(addr=h2_addr, subnet=h2_subnet)
    h2_config_thread = h2.execute_thread(host_config_script)
    config_threads[h2] = h2_config_thread

    R1 = slice.get_node(name=R1_name)  
    R1.execute("sudo sed -i '1s/.*/127.0.0.1 localhost R1/' /etc/hosts")
    if type(ip_address(R1.get_management_ip())) is IPv6Address:
        R1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")   
    R1_os_iface = R1.get_interface(network_name=net_h1_name)
    R1_os_iface.ip_addr_add(addr=R1_addr1, subnet=h1_subnet)
    R1_os_iface = R1.get_interface(network_name=net_R1_R2_name)
    R1_os_iface.ip_addr_add(addr=R1_addr2, subnet=R1_subnet)
    R1_config_thread = R1.execute_thread(host_config_script)
    config_threads[R1] = R1_config_thread

    R2 = slice.get_node(name=R2_name) 
    R2.execute("sudo sed -i '1s/.*/127.0.0.1 localhost R2/' /etc/hosts")
    if type(ip_address(R2.get_management_ip())) is IPv6Address:
        R2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")   
    R2_os_iface = R2.get_interface(network_name=net_h2_name)
    R2_os_iface.ip_addr_add(addr=R2_addr1, subnet=h2_subnet)   
    R2_os_iface = R2.get_interface(network_name=net_R1_R2_name)
    R2_os_iface.ip_addr_add(addr=R2_addr2, subnet=R2_subnet)
    R2_config_thread = R2.execute_thread(host_config_script)
    config_threads[R2] = R2_config_thread
    print("Packages and tools are now installed")

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()



Reading package lists...
Building dependency tree...
Reading state information...
0 upgraded, 0 newly installed, 0 to remove and 161 not upgraded.


Reading package lists...
Building dependency tree...
Reading state information...
0 upgraded, 0 newly installed, 0 to remove and 160 not upgraded.
Packages and tools are now installed


### 4.2 Clone the project repository on h1

Clones the experiment repository that includes topology scripts and helper utilities.


In [10]:
h1.execute("git clone https://github.com/gomezgaona/AQM_CCA.git")

Cloning into 'AQM_CCA'...


('', "Cloning into 'AQM_CCA'...\n")

### 4.3 Install Mininet

Installs Mininet from source on **h1** (and any required system dependencies).


In [11]:
h1.execute("git clone https://github.com/mininet/mininet &> /dev/null")
h1.execute("cd mininet && git checkout master &> /dev/null")
h1.execute("sudo ./util/install.sh -a &> /dev/null")

h2.execute("git clone https://github.com/mininet/mininet &> /dev/null")
h2.execute("cd mininet && git checkout master &> /dev/null")
h2.execute("sudo ./util/install.sh -a &> /dev/null")


('', '')

### 4.4 Configure routing between FABRIC VMs

Adds routes on **h1** and **h2** and configures static routes on **R1** and **R2** so the two sides can reach each other through the routers.


In [12]:
h1.execute("sudo ifconfig enp7s0 up")
h1.execute("sudo ip route add 172.17.0.0/16 via 172.16.255.254")
h2.execute("sudo ifconfig enp6s0 up")
h2.execute("sudo ip route add 172.16.0.0/16 via 172.17.255.254")

R1.execute("sudo sysctl -w net.ipv4.conf.all.forwarding=1")
R2.execute("sudo sysctl -w net.ipv4.conf.all.forwarding=1")

R1.execute("sudo ip route add 172.17.0.0/16 via 192.168.12.2")
R2.execute("sudo ip route add 172.16.0.0/16 via 192.168.12.1")

sudo: ifconfig: command not found
ip: error while loading shared libraries: libbpf.so.0: cannot open shared object file: No such file or directory
net.ipv4.conf.all.forwarding = 1
net.ipv4.conf.all.forwarding = 1


('', '')

### 4.5 Measure baseline RTT (used for BDP sizing)

Pings across the router link to estimate the baseline RTT.
The next section uses this value to compute buffer sizing or shaping latency targets.


In [13]:
import re

def _to_text(output) -> str:
    """Convert FABRIC execute() output to a single text string."""
    if isinstance(output, (list, tuple)):
        return "\n".join(str(x) for x in output)
    return str(output)

def extract_avg_rtt_ms(ping_output) -> float:
    """
    Parse ping output and return avg RTT in ms.
    Handles Linux 'ping' summary lines like:
      rtt min/avg/max/mdev = 0.123/0.456/0.789/0.012 ms
      round-trip min/avg/max/stddev = ... ms
    """
    text = _to_text(ping_output)

    pattern = r"(?:rtt|round-trip).*=\s*([\d\.]+)/([\d\.]+)/([\d\.]+)/([\d\.]+)\s*ms"
    m = re.search(pattern, text)
    if not m:
        raise ValueError(f"Could not find RTT summary line. Output was:\n{text}")

    return float(m.group(2))  # avg

def measure_avg_rtt(R1, dst_ip="192.168.12.2", count=10) -> float:
    out = R1.execute(f"ping -c {count} {dst_ip}")
    return extract_avg_rtt_ms(out)

# Usage
avg = measure_avg_rtt(R1, "192.168.12.2", 10)
print(f"Average RTT: {avg:.3f} ms")

K_BDP = 10  # buffer size in BDP
base_rtt_ms = avg  # better: min RTT

latency_ms = K_BDP * base_rtt_ms
burst = "5000000"     #10,000,000,000/250

R1.execute(f"sudo tc qdisc replace dev enp7s0 root tbf rate 10gbit burst {burst} latency {latency_ms}ms")
R1.execute("sudo tc qdisc show dev enp7s0 root")
R1.execute(f"sudo tc qdisc replace dev enp8s0 root tbf rate 10gbit burst {burst} latency {latency_ms}ms")
R1.execute("sudo tc qdisc show dev enp8s0 root")

PING 192.168.12.2 (192.168.12.2) 56(84) bytes of data.
64 bytes from 192.168.12.2: icmp_seq=1 ttl=64 time=129 ms
64 bytes from 192.168.12.2: icmp_seq=2 ttl=64 time=64.4 ms
64 bytes from 192.168.12.2: icmp_seq=3 ttl=64 time=64.4 ms
64 bytes from 192.168.12.2: icmp_seq=4 ttl=64 time=64.3 ms
64 bytes from 192.168.12.2: icmp_seq=5 ttl=64 time=64.4 ms
64 bytes from 192.168.12.2: icmp_seq=6 ttl=64 time=64.3 ms
64 bytes from 192.168.12.2: icmp_seq=7 ttl=64 time=64.3 ms
64 bytes from 192.168.12.2: icmp_seq=8 ttl=64 time=64.3 ms
64 bytes from 192.168.12.2: icmp_seq=9 ttl=64 time=64.3 ms
64 bytes from 192.168.12.2: icmp_seq=10 ttl=64 time=64.3 ms

--- 192.168.12.2 ping statistics ---
10 packets transmitted, 10 received, 0% packet loss, time 9013ms
rtt min/avg/max/mdev = 64.321/70.797/128.839/19.347 ms
Average RTT: 70.797 ms
qdisc tbf 8001: root refcnt 41 rate 10Gbit burst 4998750b lat 708ms 
qdisc tbf 8002: root refcnt 41 rate 10Gbit burst 4998750b lat 708ms 


('qdisc tbf 8002: root refcnt 41 rate 10Gbit burst 4998750b lat 708ms \n', '')

### 4.6 Tune TCP send and receive buffers (optional)

Sets host TCP buffer limits when you expect large BDP paths or many parallel flows.


In [14]:
h1.execute("sudo sysctl -w net.ipv4.tcp_wmem=\'1024 87380 200000000\'")
h1.execute("sudo sysctl -w net.ipv4.tcp_rmem=\'1024 87380 200000000\'")

h2.execute("sudo sysctl -w net.ipv4.tcp_wmem=\'1024 87380 200000000\'")
h1.execute("sudo sysctl -w net.ipv4.tcp_rmem=\'1024 87380 200000000\'")

net.ipv4.tcp_wmem = 1024 87380 200000000
net.ipv4.tcp_rmem = 1024 87380 200000000
net.ipv4.tcp_wmem = 1024 87380 200000000
net.ipv4.tcp_rmem = 1024 87380 200000000


('net.ipv4.tcp_rmem = 1024 87380 200000000\n', '')

## 5. Run Mininet and execute experiments

This section launches the Mininet topologies on **h1** and **h2**, configures the Mininet hosts, runs iPerf3, collects results, and (optionally) monitors TCP state.


In [15]:
num_hosts=16

In [16]:
#h1 = slice.get_node(name=h1_name)
#h2 = slice.get_node(name=h2_name)

h1.upload_file('scripts/topo_h1.py', 'topo_h1.py')
h2.upload_file('scripts/topo_h2.py', 'topo_h2.py')
R1.upload_file('scripts/log_qdisc_enp8s0.py', 'log_qdisc_enp8s0.py')

#h1.execute('sudo mn -c &> /dev/null')
#h2.execute('sudo mn -c &> /dev/null')

h1.execute('mkdir results')
h2.execute('mkdir results')

#h1.execute(f'sudo python3 topo_h1.py {num_hosts} &')
#h2.execute(f'sudo python3 topo_h2.py {num_hosts} &')

('', '')

### 5.1 Get SSH commands for each node

Use the printed SSH commands to open terminals to **h1**, **h2**, **R1**, and **R2** when you need to run commands interactively.


In [17]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

R1: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fee6:96b5
R2: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe0f:fa79
h1: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe25:e031
h2: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe31:fe2d


### 5.2 Launch the Mininet topologies manually on the VMs

Run the topology scripts on **h1** and **h2** using SSH (recommended).
This keeps long-running Mininet processes stable even if the notebook kernel disconnects.


In [18]:
print(f"Run the following command on h1: sudo python3 topo_h1.py {num_hosts}")
print(f"Run the following command on h2: sudo python3 topo_h2.py {num_hosts}")

Run the following command on h1: sudo python3 topo_h1.py 16
Run the following command on h2: sudo python3 topo_h2.py 16


### 5.3 Attach VM interfaces to the Mininet switches

Adds the VM physical interfaces into the OVS switches created by the topology scripts.
Interface names are environment specific, so verify with `ip link` if a command fails.


In [20]:
h1.execute('sudo ovs-vsctl add-port s_agg1 enp7s0')#Interfaces are hardcoded
h2.execute('sudo ovs-vsctl add-port s_agg2 enp6s0')

('', '')

### 5.4 Configure routing inside the Mininet hosts

Adds routes on each Mininet sender and receiver so flows traverse the router path.
This step also sets the initial congestion window (`initcwnd`) when specified.


In [21]:
for i in range(1, num_hosts + 1):
    print(f"Setting default route on host hs{i} and hr{i}")
    h1.execute(f"mininet/util/m hs{i} ip route add 172.17.0.0/16 via 172.16.255.254 dev hs{i}-eth0 initcwnd 100000 &> /dev/null")
    h2.execute(f"mininet/util/m hr{i} ip route add 172.16.0.0/16 via 172.17.255.254 dev hr{i}-eth0 initcwnd 100000 &> /dev/null")
print(f"Default routes configured in sender hosts (hs) and receiver hosts (hr)")

Setting default route on host hs1 and hr1
Setting default route on host hs2 and hr2
Setting default route on host hs3 and hr3
Setting default route on host hs4 and hr4
Setting default route on host hs5 and hr5
Setting default route on host hs6 and hr6
Setting default route on host hs7 and hr7
Setting default route on host hs8 and hr8
Setting default route on host hs9 and hr9
Setting default route on host hs10 and hr10
Setting default route on host hs11 and hr11
Setting default route on host hs12 and hr12
Setting default route on host hs13 and hr13
Setting default route on host hs14 and hr14
Setting default route on host hs15 and hr15
Setting default route on host hs16 and hr16
Default routes configured in sender hosts (hs) and receiver hosts (hr)


### 5.5 Populate ARP tables

Runs one ping per host to prime neighbor entries before starting measurements.


In [22]:
for i in range(1, num_hosts + 1):
    h1.execute(f"mininet/util/m hs{i} ping 172.17.0.{i} -c 1")
    h2.execute(f"mininet/util/m hr{i} ping 172.16.0.{i} -c 1")
print(f"ARP tables populated in sender hosts (hs) and receiver hosts (hr)")

PING 172.17.0.1 (172.17.0.1) 56(84) bytes of data.

--- 172.17.0.1 ping statistics ---
1 packets transmitted, 0 received, 100% packet loss, time 0ms

PING 172.16.0.1 (172.16.0.1) 56(84) bytes of data.
From 172.17.0.1 icmp_seq=1 Destination Host Unreachable

--- 172.16.0.1 ping statistics ---
1 packets transmitted, 0 received, +1 errors, 100% packet loss, time 0ms

PING 172.17.0.2 (172.17.0.2) 56(84) bytes of data.

--- 172.17.0.2 ping statistics ---
1 packets transmitted, 0 received, 100% packet loss, time 0ms

PING 172.16.0.2 (172.16.0.2) 56(84) bytes of data.
From 172.17.0.2 icmp_seq=1 Destination Host Unreachable

--- 172.16.0.2 ping statistics ---
1 packets transmitted, 0 received, +1 errors, 100% packet loss, time 0ms

PING 172.17.0.3 (172.17.0.3) 56(84) bytes of data.

--- 172.17.0.3 ping statistics ---
1 packets transmitted, 0 received, 100% packet loss, time 0ms

PING 172.16.0.3 (172.16.0.3) 56(84) bytes of data.
From 172.17.0.3 icmp_seq=1 Destination Host Unreachable

--- 172.

### 5.6 Start iPerf3 servers (receivers)

Starts one iPerf3 server per receiver host on **h2**.


In [23]:
for i in range(1, num_hosts + 1):
    print(f"Starting iperf3 server on host hr{i}")
    h2.execute_thread(f"mininet/util/m hr{i} iperf3 -s")

Starting iperf3 server on host hr1
Starting iperf3 server on host hr2
Starting iperf3 server on host hr3
Starting iperf3 server on host hr4
Starting iperf3 server on host hr5
Starting iperf3 server on host hr6
Starting iperf3 server on host hr7
Starting iperf3 server on host hr8
Starting iperf3 server on host hr9
Starting iperf3 server on host hr10
Starting iperf3 server on host hr11
Starting iperf3 server on host hr12
Starting iperf3 server on host hr13
Starting iperf3 server on host hr14
Starting iperf3 server on host hr15
Starting iperf3 server on host hr16


### 5.7 Run the experiment

Configures the bottleneck (buffering or shaping), then starts iPerf3 clients on the sender hosts.
Update the client command to match your transfer size, duration, and congestion control algorithm.


#### 5.7.1 Setting router R1 buffer size


In [ ]:
K_BDP = 10  # buffer size in BDP
base_rtt_ms = avg  # better: min RTT

latency_ms = K_BDP * base_rtt_ms
burst = "5000000"     #10,000,000,000/250

R1.execute(f"sudo tc qdisc replace dev enp7s0 root tbf rate 10gbit burst {burst} latency {latency_ms}ms")
R1.execute("sudo tc qdisc show dev enp7s0 root")


#### 5.7.2 Disabling TCP offloading


In [ ]:
for i in range(1, num_hosts + 1):
    print(f"Configuring host hs{i} and hr{i} (offloads, qdisc, ECN, CCA)")
    h1.execute(f"mininet/util/m hs{i} sudo ethtool -K hs{i}-eth0 tso off gso off gro off lro off")
    h1.execute(f"mininet/util/m hs{i} sudo tc qdisc replace dev hs{i}-eth0 root handle 1: fq limit 20480 flow_limit 10240")
    h1.execute(f"mininet/util/m hs{i} sysctl -w net.ipv4.tcp_ecn=3")
    h1.execute(f"mininet/util/m hs{i} sysctl -w net.ipv4.tcp_congestion_control=prague")
    
    h2.execute(f"mininet/util/m hr{i} sudo ethtool -K hr{i}-eth0 tso off gso off gro off lro off")
    h2.execute(f"mininet/util/m hr{i} sudo tc qdisc replace dev hr{i}-eth0 root handle 1: fq limit 20480 flow_limit 10240")
    h2.execute(f"mininet/util/m hr{i} sysctl -w net.ipv4.tcp_ecn=3")
    h2.execute(f"mininet/util/m hr{i} sysctl -w net.ipv4.tcp_congestion_control=prague")


#### 5.7.3 Start iPerf3 clients (senders)

Runs iPerf3 from each sender host to its paired receiver and writes JSON output files under `results/`.


In [ ]:
import time

def countdown(seconds: int, prefix: str = "iPerf running"):
    """Simple MM:SS countdown that updates on the same terminal line."""
    for remaining in range(seconds, 0, -1):
        mm, ss = divmod(remaining, 60)
        print(f"\r{prefix}: {mm:02d}:{ss:02d} remaining", end="", flush=True)
        time.sleep(1)
    print(f"\r{prefix}: 00:00 remaining")

transfer_duration = 120
threads = []
num_hosts = 16
# Start flows (recommended: do NOT add '&' if you want execute_thread to manage the lifecycle)
for i in range(1, num_hosts + 1):
#    cmd = (
#        f"mininet/util/m hs{i} "
#        f"iperf3 -c 172.17.0.{i} -J -t {transfer_duration} "
#        f"> results/hs{i}_out.json"
#    )
    cmd = (
        f"mininet/util/m hs{i} "
        f"iperf3 -c 172.17.0.{i} -J -n 6.25g -C prague"
        f"> results/hs{i}_out.json"
    )
    threads.append(h1.execute_thread(cmd))

# Visual timer while tests run
#countdown(transfer_duration)

#### 5.7.2 Monitor TCP state with `ss -tin` (optional)

Parses `ss -tin` output periodically so you can observe cwnd, pacing, RTT, delivery rate, and other signals while transfers run.


In [ ]:
import re
import time

# -------------------- CONFIG --------------------
NUM_HOSTS   = 16
HOST_PREFIX = "hs"     # "hs" or "hr"
IPERF_PORT  = 5201     # set None to disable port filter
REFRESH_S   = 0.1
# -----------------------------------------------


# -------------------- EXEC (quiet if possible) --------------------
def exec_quiet(node, cmd):
    """
    Some FABRIC helpers support quiet/hide flags. Try them, otherwise fallback.
    """
    for kw in ("quiet", "hide", "suppress_output", "output"):
        try:
            if kw == "output":
                return node.execute(cmd, output=False)
            return node.execute(cmd, **{kw: True})
        except TypeError:
            pass
    return node.execute(cmd)


def _to_text(x):
    if isinstance(x, (tuple, list)):
        return "\n".join(str(p) for p in x if p is not None)
    return str(x)


# -------------------- PARSING HELPERS --------------------
def _find_int(text, key):
    m = re.search(rf"\b{key}:(\d+)\b", text)
    return int(m.group(1)) if m else None

def _find_float(text, key):
    m = re.search(rf"\b{key}:(\d+(?:\.\d+)?)\b", text)
    return float(m.group(1)) if m else None

def _find_rtt(text):
    # rtt:22.876/10.763
    m = re.search(r"\brtt:(\d+(?:\.\d+)?)/(\d+(?:\.\d+)?)\b", text)
    if not m:
        return None, None
    return float(m.group(1)), float(m.group(2))

def _find_rate_bps(text, key):
    # send 8988984bps, pacing_rate 22604480bps, delivery_rate 794488bps
    m = re.search(rf"\b{key}\s+(\d+)([KMG]?)bps\b", text)
    if not m:
        return None
    val = float(m.group(1))
    unit = m.group(2).upper()
    scale = {"": 1.0, "K": 1e3, "M": 1e6, "G": 1e9}[unit]
    return val * scale

def _find_bbr_bw_bps(text):
    # bbr:(bw:793952bps,mrtt:14.379,...)
    m = re.search(r"bbr:\(.*?\bbw:(\d+)([KMG]?)bps", text)
    if not m:
        return None
    val = float(m.group(1))
    unit = m.group(2).upper()
    scale = {"": 1.0, "K": 1e3, "M": 1e6, "G": 1e9}[unit]
    return val * scale

def _extract_ipport_tokens(header_line):
    parts = header_line.split()
    return [p for p in parts if ":" in p and re.search(r":\d+$", p)]

def _keep_port(src, dst, port):
    if port is None:
        return True
    return (src and src.endswith(f":{port}")) or (dst and dst.endswith(f":{port}"))


def parse_ss_tin_output(ss_text, port_filter=None):
    """
    Returns a list of flow dicts for ESTAB sockets.
    Each socket is usually two lines: ESTAB header + tcp info.
    """
    lines = [ln.strip() for ln in ss_text.splitlines() if ln.strip()]
    flows = []
    i = 0

    while i < len(lines):
        if not lines[i].startswith("ESTAB"):
            i += 1
            continue

        header = lines[i]
        info = lines[i + 1] if i + 1 < len(lines) else ""

        ipports = _extract_ipport_tokens(header)
        src = ipports[0] if len(ipports) >= 1 else None
        dst = ipports[1] if len(ipports) >= 2 else None

        if not _keep_port(src, dst, port_filter):
            i += 2
            continue

        tokens = info.split()
        cc = tokens[0] if tokens else None

        rtt_ms, _ = _find_rtt(info)

        flows.append({
            "src": src,
            "dst": dst,
            "cc": cc,
            "rtt_ms": rtt_ms,
            "minrtt_ms": _find_float(info, "minrtt"),
            "cwnd": _find_int(info, "cwnd"),
            "mss": _find_int(info, "mss"),
            "rto": _find_int(info, "rto"),              # usually ms (no unit)
            "bytes_acked": _find_int(info, "bytes_acked"),
            "send_bps": _find_rate_bps(info, "send"),
            "pacing_bps": _find_rate_bps(info, "pacing_rate"),
            "delivery_bps": _find_rate_bps(info, "delivery_rate"),
            "bbr_bw_bps": _find_bbr_bw_bps(info),
        })

        i += 2

    return flows


# -------------------- SUMMARIZATION --------------------
def mean(vals):
    vals = [v for v in vals if v is not None]
    return (sum(vals) / len(vals)) if vals else None

def sum_or_none(vals):
    vals = [v for v in vals if v is not None]
    return sum(vals) if vals else None

def bps_to_mbps(x):
    return None if x is None else (x / 1e6)

def fmt_float(x, w=9, p=2):
    return f"{'-':>{w}}" if x is None else f"{x:>{w}.{p}f}"

def fmt_int(x, w=6):
    return f"{'-':>{w}}" if x is None else f"{int(x):>{w}}"

def fmt_bps(x, w=12):
    # scientific keeps table narrow but still "bps"
    return f"{'-':>{w}}" if x is None else f"{x:>{w}.3e}"

def dominant_cc(ccs):
    ccs = [c for c in ccs if c]
    if not ccs:
        return "-"
    return max(set(ccs), key=ccs.count)

def summarize_host(flows, ack_rate_bps):
    send_bps = sum_or_none([f["send_bps"] for f in flows])
    pacing_bps = mean([f["pacing_bps"] for f in flows])
    delivery_bps = mean([f["delivery_bps"] for f in flows])
    bbr_bw_bps = mean([f["bbr_bw_bps"] for f in flows])

    return {
        "flows": len(flows),
        "cc": dominant_cc([f["cc"] for f in flows]),
        "avg_rtt": mean([f["rtt_ms"] for f in flows]),
        "min_rtt": mean([f["minrtt_ms"] for f in flows]),
        "avg_cwnd": mean([f["cwnd"] for f in flows]),
        "mss": mean([f["mss"] for f in flows]),
        "rto": mean([f["rto"] for f in flows]),
        # outgoing (from ss instantaneous rates)
        "send_mbps": bps_to_mbps(send_bps),
        "send_bps": send_bps,
        "pacing_mbps": bps_to_mbps(pacing_bps),
        "pacing_bps": pacing_bps,
        "delivery_mbps": bps_to_mbps(delivery_bps),
        "delivery_bps": delivery_bps,
        "bbr_bw_mbps": bps_to_mbps(bbr_bw_bps),
        "bbr_bw_bps": bbr_bw_bps,
        # incoming “ACK progress rate”: delta(bytes_acked)/dt * 8
        "ack_mbps": bps_to_mbps(ack_rate_bps),
        "ack_bps": ack_rate_bps,
    }


# -------------------- TABLE ONLY OUTPUT --------------------
def clear_only_table():
    # Best for notebooks; does not print extra text.
    try:
        from IPython.display import clear_output
        clear_output(wait=True)
    except Exception:
        print("\033[2J\033[H", end="")

def print_table(rows):
    header = (
        f"{'Host':<5} {'#F':>3} {'CC':<5} "
        f"{'RTT':>7} {'minRTT':>7} {'cwnd':>6} {'MSS':>5} {'RTO':>5} "
        f"{'Send(M)':>9} {'Send(bps)':>12} "
        f"{'ACK(M)':>9} {'ACK(bps)':>12} "
        f"{'Pace(M)':>9} {'Del(M)':>9} {'BBRbw(M)':>9}"
    )
    print(header)
    print("-" * len(header))

    for r in rows:
        print(
            f"{r['host']:<5} {r['flows']:>3} {r['cc']:<5} "
            f"{fmt_float(r['avg_rtt'],7,2)} {fmt_float(r['min_rtt'],7,2)} "
            f"{fmt_float(r['avg_cwnd'],6,1)} {fmt_int(r['mss'],5)} {fmt_float(r['rto'],5,0)} "
            f"{fmt_float(r['send_mbps'],9,2)} {fmt_bps(r['send_bps'],12)} "
            f"{fmt_float(r['ack_mbps'],9,2)} {fmt_bps(r['ack_bps'],12)} "
            f"{fmt_float(r['pacing_mbps'],9,2)} {fmt_float(r['delivery_mbps'],9,2)} {fmt_float(r['bbr_bw_mbps'],9,2)}"
        )


# -------------------- MAIN MONITOR LOOP --------------------
def monitor_ss_table(h1):
    prev_acked = {}  # host -> last total bytes_acked
    prev_time = {}   # host -> timestamp

    while True:
        rows = []
        now = time.time()

        for i in range(1, NUM_HOSTS + 1):
            host = f"{HOST_PREFIX}{i}"
            cmd = f"mininet/util/m {host} ss -tin"

            out = exec_quiet(h1, cmd)
            text = _to_text(out)
            flows = parse_ss_tin_output(text, port_filter=IPERF_PORT)

            # --- Incoming "ACK progress rate" (acked bytes per second) ---
            total_bytes_acked = sum_or_none([f["bytes_acked"] for f in flows]) or 0
            last_bytes = prev_acked.get(host, total_bytes_acked)
            last_t = prev_time.get(host, now)

            dt = max(1e-6, now - last_t)
            delta_bytes = max(0, total_bytes_acked - last_bytes)
            ack_rate_bps = (delta_bytes * 8.0) / dt  # bits/s

            prev_acked[host] = total_bytes_acked
            prev_time[host] = now

            summary = summarize_host(flows, ack_rate_bps)
            summary["host"] = host
            rows.append(summary)

        clear_only_table()
        print_table(rows)
        time.sleep(REFRESH_S)


# -------- Run in your notebook after h1 is defined --------
monitor_ss_table(h1)


#### 5.7.3 Download JSON result files

Copies iPerf3 JSON files from **h1** to the notebook working directory for plotting.


In [ ]:
for i in range(1, num_hosts + 1):
    h1.download_file(f"results/hs{i}_out.json",f"results/hs{i}_out.json")
print("JSON files are now ready for plotting!")

#### 5.7.4 Archive results to a ZIP file (optional)

Zips all JSON results under `results/` and moves the archive into `archived_experiments/`.


In [ ]:
#!/usr/bin/env python3
# Zip all .json files in ./results and move the zip to ./archived_experiments

from datetime import datetime
from pathlib import Path
from zipfile import ZipFile, ZIP_DEFLATED


# --- Paths (relative to where you run the script) ---
BASE_DIR = Path.cwd()                 # should be .../AQM_CCA when you run it there
RESULTS_DIR = BASE_DIR / "results"
ARCHIVE_DIR = BASE_DIR / "archived_experiments"

# --- Hardcode your label pieces here ---
EXP_NAME = "preliminary"
NUM_FLOWS = 16
CC_NAME = "bbr3"
BUF_BDP = K_BDP


def build_label() -> str:
    return f"{EXP_NAME}_{NUM_FLOWS}f_{CC_NAME}_{BUF_BDP}bdp"


def create_zip(label: str) -> Path:
    today = datetime.now().strftime("%Y%m%d")
    zip_path = RESULTS_DIR / f"{today}_{label}.zip"

    json_files = sorted(RESULTS_DIR.glob("*.json"))
    if not json_files:
        raise FileNotFoundError(f"No .json files found in {RESULTS_DIR.resolve()}")

    with ZipFile(zip_path, "w", compression=ZIP_DEFLATED) as zf:
        for f in json_files:
            zf.write(f, arcname=f.name)

    return zip_path


def move_to_archive(zip_path: Path) -> Path:
    if not ARCHIVE_DIR.is_dir():
        raise FileNotFoundError(f"Archive directory not found: {ARCHIVE_DIR.resolve()}")

    dest = ARCHIVE_DIR / zip_path.name
    return zip_path.replace(dest)   # move/rename


def main():
    label = build_label()

    zip_path = create_zip(label)
    archived_path = move_to_archive(zip_path)

    print(f"Archived zip: {archived_path}")


if __name__ == "__main__":
    main()


## 6. Plotting and analysis

This section reads the iPerf3 JSON outputs and produces plots for throughput over time.


### 6.1 Throughput as a function of time

Edit the experiment labels (name, CCA, buffer settings) to match the run you want to plot.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import matplotlib
import math
from datetime import datetime
from pathlib import Path

EXP_NAME = "preliminary"
CC_NAME  = "bbr3"
BUF_BDP  = 32

RESULTS_DIR = Path("results")
PLOTS_DIR   = Path("plots")   # <-- so it becomes AQM_CCA/plots (not AQM_CCA/AQM_CCA/plots)

def extract_iperf_timming(filename):
    with open(filename) as f:
        data = json.load(f)
    intervals = data.get("intervals", [])
    return [float(it["sum"]["bits_per_second"]) / 1e9 for it in intervals]

def setup_plot(font_size=12):
    font = {'family': 'normal', 'weight': 'normal', 'size': font_size}
    matplotlib.rc('font', **font)
    fig, axes = plt.subplots(3, 1, sharex=True, figsize=(14, 8))
    fig.subplots_adjust(hspace=0.1)
    for ax in axes:
        ax.grid(True, which="both", lw=0.3, linestyle=(0, (1, 10)), color='black')
    return fig, axes

def calculate_fairness(th_per_flow):
    th = np.asarray(th_per_flow, dtype=float)
    m, _ = th.shape
    sum_x = th.sum(axis=0)
    sum_x2 = (th ** 2).sum(axis=0)
    denom = m * sum_x2
    fairness = np.where(denom == 0, 100.0, 100.0 * (sum_x ** 2) / denom)
    return fairness.tolist()

def build_pdf_name():
    today = datetime.now().strftime("%Y%m%d")
    return f"{today}_{EXP_NAME}_{num_hosts}f_{CC_NAME}_{BUF_BDP}bdp.pdf"

def plot_results(throughput, th_per_flow, out_path):
    fairness = calculate_fairness(th_per_flow)
    fig, axes = setup_plot()

    N = len(throughput)
    t = list(range(N))

    axes[0].plot(t, fairness, '#D4AF37', linewidth=2, label='Fairness', marker='o')
    axes[1].plot(t, throughput, '#2D72B7', linewidth=2, label='Agg. Tput', marker='o')

    for flow_id, flow_series in enumerate(th_per_flow):
        axes[2].plot(t, flow_series[:N], linewidth=1.5, marker='o', label=f'Flow {flow_id+1}')

    axes[0].set_ylabel('Fairness [%]')
    axes[1].set_ylabel('Throughput [Gbps]')
    axes[2].set_ylabel('Throughput [Gbps]')
    axes[2].set_xlabel('Time [seconds]')

    axes[0].legend(loc="lower right")
    axes[1].legend(loc="upper right")

    num_flows = len(th_per_flow)
    items_per_col = 2
    ncol = max(1, math.ceil(num_flows / items_per_col))
    axes[2].legend(loc="upper right", ncol=ncol, fontsize=10, frameon=True)

    axes[0].set_ylim([0, 105])
    axes[1].set_ylim(0, float(np.max(throughput)) + 5)
    axes[2].set_ylim(0, float(np.max(th_per_flow)) + 5)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_path, bbox_inches='tight')

    # IMPORTANT: show BEFORE closing
    plt.show()
    plt.close(fig)

    print(f"Saved PDF: {out_path.resolve()}")

def main():
    th_per_flow = []
    for i in range(1, num_hosts + 1):
        th_per_flow.append(extract_iperf_timming(RESULTS_DIR / f"hs{i}_out.json"))

    N = min(len(x) for x in th_per_flow)
    th_per_flow = [x[:N] for x in th_per_flow]

    M = len(th_per_flow)
    agg_th = [0.0] * N
    for t in range(N):
        total = 0.0
        for flow in range(M):
            total += th_per_flow[flow][t]
        agg_th[t] = total

    out_pdf = PLOTS_DIR / build_pdf_name()
    plot_results(agg_th, th_per_flow, out_pdf)

if __name__ == '__main__':
    main()


## Appendix: FABlib inspection helpers (optional)

Convenience cells to inspect sites, slice details, nodes, and interfaces during debugging.


### A.1 List available sites


In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

### A.2 Print slice attributes


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### A.3 List nodes


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

### A.4 Print node details


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

### A.5 List interfaces


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## 7. Cleanup

Delete the slice when you are done to release resources.


In [ ]:
slice.delete()